In [18]:
import os
import gzip
import json
from collections import defaultdict, OrderedDict

import numpy as np
import pandas as pd
import spacy

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
nlp_uk = spacy.load('/mnt/slow_drive/dmytro/uk_vectors')

In [3]:
base_dir = './1551.gov.ua/raw/'

file_list = []
for letter in os.listdir(base_dir):
    for filename in os.listdir(base_dir + letter):
        file_list.append(base_dir + letter + '/' + filename)

call_data = []

for filename in file_list:
    with gzip.open(filename, 'r') as zipfile:
        call_info = json.load(zipfile)
        call_data.append((call_info[0]['CallZText'], call_info[0]['CallZType']))

'Кількість даних і приклад', len(call_data), call_data[0]

('Кількість даних і приклад',
 127329,
 ('Досі не має теплозабезпечення по нашій адресі.На всі запити нам ЖКХ відповідае,що це не їх проблема,щоб ми зверталися в Київенерго.В нас маленька дитина хворіе ніяк не можимо вилікувати,а батареї холодні і температура в приміщені 14 градусїв.Велике прохання допомогти в рішені проблеми ,так як отоплювальний сезон вже розпочався.',
  'Відсутність опалення'))

In [4]:
categories = set([c[1] for c in call_data])
'Кількість унікальних категорій {}'.format(len(categories))

'Кількість унікальних категорій 920'

In [5]:
from category_changes import clean_call_data

cleaned_call_data = clean_call_data(call_data)
cleaned_categories = set([c[2] for c in cleaned_call_data])
print('Кількість унікальних категорій: {}'.format(len(cleaned_categories)))
call_df = pd.DataFrame(cleaned_call_data)
call_df.columns = ['text', 'category', 'new_category']
call_df

Кількість унікальних категорій: 758


,text,category,new_category
0,Досі не має теплозабезпечення по нашій адресі....,Відсутність опалення,Відсутність опалення
1,Ініціативною групою мешканців нашого будинку б...,Відсутність освітлення у під’їзді за відсутнос...,Відсутність освітлення у під’їзді за відсутнос...
2,Жахливий стан дорожнього покриття на підїзді д...,Укладання та ремонт асфальтного покриття,Укладання та ремонт асфальтного покриття
3,Не получил ответа от Жуковой Ларисы Петровны п...,"Пропозиції, зауваження, щодо роботи web-сторін...","Пропозиції, зауваження, щодо роботи web-сторін..."
4,"Подъездная дорога с ул.Белорусская к домам #5,...",Укладання та ремонт асфальтного покриття,Укладання та ремонт асфальтного покриття
...,...,...,...
127210,Остатки велодорожек.,Облаштування велосипедних доріжок,Облаштування велосипедних доріжок
127211,В доме № 33 на улице Новопироговской не включе...,Відсутність опалення,Відсутність опалення
127212,"Навпроти нашого будинку (вул. Сортувальна, 4) ...",Облаштування наземного пішохідного переходу,Облаштування наземного пішохідного переходу
127213,Трафареты готовы !!! Помогите краской ! И мы с...,Облаштування велосипедних доріжок,Облаштування велосипедних доріжок


In [6]:
digit_to_cat = {i: category for i, category in enumerate(cleaned_categories)}
cat_to_digit = {category: i for i, category in digit_to_cat.items()}

call_data = []
for row in cleaned_call_data:
    call_data.append((row[0], cat_to_digit.get(row[2])))

call_df = pd.DataFrame(call_data)
call_df.columns = ['text', 'digit']
call_df

,text,digit
0,Досі не має теплозабезпечення по нашій адресі....,261
1,Ініціативною групою мешканців нашого будинку б...,110
2,Жахливий стан дорожнього покриття на підїзді д...,255
3,Не получил ответа от Жуковой Ларисы Петровны п...,50
4,"Подъездная дорога с ул.Белорусская к домам #5,...",255
...,...,...
127210,Остатки велодорожек.,364
127211,В доме № 33 на улице Новопироговской не включе...,261
127212,"Навпроти нашого будинку (вул. Сортувальна, 4) ...",325
127213,Трафареты готовы !!! Помогите краской ! И мы с...,364


In [120]:
y_column = 'digit'
copy_df = call_df.copy()
y = list(copy_df.loc[:, y_column])
copy_df.drop([y_column], axis=1, inplace=True)

X = [v['text'] for v in copy_df.T.to_dict().values()]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8, test_size=0.2, random_state=0, shuffle=True, stratify=y
)

In [121]:
X_train2, X_val2, y_train2, y_val2 = train_test_split(
    X_train, y_train,
    train_size=0.8, test_size=0.2, random_state=0, shuffle=True, stratify=y_train
)

In [122]:
len(X_train), len(X_test), len(X_train2), len(X_val2)

(101772, 25443, 81417, 20355)

In [123]:
X_test[0], y_test[0]

('Вы что там ? Что делить  в ”зверненні” ? Каждую бумажку, что ли отдельно расписывать? Повторяю в ТРЕТИЙ РАЗ !!!!!!!!!! Жалоба не неубранный подъезд : нет ни влажной уборки, ни теперь уже не подметается неделю,  в лифте очень грязный пол! Перед мусоросборниками грязь и вонь , тараканы!!!!!!!! Что здесь разделять???????????  Это , что не услуги одной категории?  Квитанции так присылаете, а оказывать услуги,  не хотим, а теперь еще отфутболивать научились? в СУД  ХОТИТЕ?',
 63)

In [124]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
# Let's make a text-only dataset (no labels):
#text_ds = raw_train_ds.map(lambda x, y: x)
text_ds = tf.data.Dataset.from_tensor_slices(X_train2).batch(32)
vectorizer.adapt(text_ds)

In [125]:
vectorizer.get_vocabulary()[:10]

['', '[OOV]', 'в', 'на', 'не', 'и', 'та', 'по', 'за', 'з']

In [126]:
output = vectorizer(np.array([["У попа була собака."]]))
output.numpy()[0, :6]

array([  46,    1,  209, 8043,    0,    0])

In [127]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(2, len(voc))))

In [128]:
embeddings_index = {}
with open('./news.cased.tokenized.word2vec.300d') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 365320 word vectors.


In [129]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 15348 words (4650 misses)


In [26]:
from tensorflow import keras
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [145]:
x_train_ = vectorizer(np.array([[s] for s in X_train2])).numpy()
x_val_ = vectorizer(np.array([[s] for s in X_val2])).numpy()
x_test_ = vectorizer(np.array([[s] for s in X_test])).numpy()

y_train_ = np.array(y_train2)
y_val_ = np.array(y_val2)
y_test_ = np.array(y_test)

In [180]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
x = embedding_layer(int_sequences_input)
x = layers.LSTM(128, return_sequences=True, activation='relu')(x)
#x = layers.Conv1D(128, 5, activation="relu")(x)
#x = layers.MaxPooling1D(5)(x)
#x = layers.Conv1D(128, 5, activation="relu")(x)
#x = layers.MaxPooling1D(5)(x)
#x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
#x = layers.Dense(128, activation="relu")(x)
#x = layers.Dropout(0.5)(x)
preds = layers.Dense(n_classes, activation="softmax")(x)
model2 = keras.Model(int_sequences_input, preds)
print(model2.summary())
model2.output_shape

Model: "functional_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        multiple                  6000600   
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 128)         219648    
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 128)               0         
_________________________________________________________________
dense_73 (Dense)             (None, 758)               97782     
Total params: 6,318,030
Trainable params: 317,430
Non-trainable params: 6,000,600
_________________________________________________________________
None


(None, 758)

In [ ]:
model2.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
epochs = 10

# Fit the model using the train and test datasets.
model2.fit(x_train_, y_train_, batch_size=32, epochs=epochs, validation_data=(x_val_, y_val_))

Epoch 1/10
2545/2545 [==============================] - 365s 143ms/step - loss: 3.0208 - accuracy: 0.3780 - val_loss: 2.8376 - val_accuracy: 0.4108
Epoch 2/10
2545/2545 [==============================] - 368s 144ms/step - loss: 2.4768 - accuracy: 0.4464 - val_loss: 2.6820 - val_accuracy: 0.4315
Epoch 3/10
2545/2545 [==============================] - 366s 144ms/step - loss: 2.1931 - accuracy: 0.4839 - val_loss: 2.6489 - val_accuracy: 0.4506
Epoch 4/10
2545/2545 [==============================] - 367s 144ms/step - loss: 2.0481 - accuracy: 0.5091 - val_loss: 3.1807 - val_accuracy: 0.3944
Epoch 5/10
2545/2545 [==============================] - 363s 142ms/step - loss: 2.0003 - accuracy: 0.5148 - val_loss: 2.7574 - val_accuracy: 0.4486
Epoch 6/10
2545/2545 [==============================] - 363s 143ms/step - loss: 1.7413 - accuracy: 0.5612 - val_loss: 2.8141 - val_accuracy: 0.4528
Epoch 7/10
2545/2545 [==============================] - 361s 142ms/step - loss: 1.6291 - accuracy: 0.5836 - val_

In [183]:
model2.fit(x_train_, y_train_, batch_size=32, epochs=1, validation_data=(x_val_, y_val_))

2545/2545 [==============================] - 358s 141ms/step - loss: 1.3399 - accuracy: 0.6404 - val_loss: 3.2006 - val_accuracy: 0.4596


In [184]:
# ^ Дотренував 11-й раз, бо відвалився VPN і ноутбук не показав, що там відбулося

y_pred = predict(model2, x_test_)
print(classification_report(y_pred, y_test_))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      1.00      1.00         1
           3       0.09      1.00      0.17         1
           4       0.14      0.30      0.20        53
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.04      0.10      0.05        10
           9       0.02      0.09      0.03        11
          10       0.00      0.00      0.00         0
          12       0.25      1.00      0.40         2
          13       0.20      0.13      0.16        47
          14       0.00      0.00      0.00         1
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         0
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         2
          19       0.00    

/home/dmytro/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Отримав macro avg 0.17, тут ще можна звичайно покращити модель, але вона так довго тренується, що якщо додам ще різних шарів, то невідомо скільки це займе і чи вкладуся я в суботу.

Отож, тут **0.17**, на моєму baselie було лишень **0.11**, а далі я в 10-й домашній роботі не просунувся.